**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self, e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

`epsilon` is essential to ensure there is at least some level of exploration during training. Otherwise, the network might get stuck in a bad policy and never explore better alternatives (which would prevent it from learning about them).

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=300 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

`board` holds the rewards that the rat will receive when he steps on each cell. `position` is a one-hot encoding of the current position of the rat on the board (with $-1$s at the borders).

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()

    def learned_act(self, s):
        return np.random.randint(self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        state = env.reset()
        game_over = False
        win = 0
        lose = 0
        while not game_over:
            action = agent.act(state)
            prev_state = state
            state, reward, game_over = env.act(action)
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e+1))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random{}.mp4'.format(epochs_test)))

Win/lose count 14.0/13.0. Average score (1.0)
Win/lose count 11.0/11.0. Average score (0.5)
Win/lose count 18.0/17.0. Average score (0.6666666666666666)
Win/lose count 8.5/16.0. Average score (-1.375)
Win/lose count 11.5/15.0. Average score (-1.8)
Win/lose count 11.5/15.0. Average score (-2.0833333333333335)
Win/lose count 10.0/11.0. Average score (-1.9285714285714286)
Win/lose count 7.5/11.0. Average score (-2.125)
Win/lose count 9.0/18.0. Average score (-2.888888888888889)
Win/lose count 9.0/16.0. Average score (-3.3)
Final score: -3.3


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




First, we have:
$$\begin{align*}
Q^\pi(s,a) &= E_{p^{\pi}}\left[ \sum_{t}\gamma^{t}r(s_{t},a_{t})\;\middle|\;s_{0}=s,a_{0}=a \right] \\
&= E_{p^{\pi}}\left[ r(s,a) + \gamma\sum_{1\leq t}\gamma^{t-1}r(s_{t},a_{t})\;\middle|\;s_{0}=s,a_{0}=a \right] \\
&= E_{(s',a')\sim p(\cdot\;|\;s,a)}\left[r(s,a)+\gamma E_{p^\pi}\left[  \sum_{t}\gamma^{t}r(s_{t},a_{t})\;\middle|\;s_{0}=s',a_{0}=a' \right]\right] \\
&= E_{(s',a')\sim p(\cdot\;|\;s,a)}\left[r(s,a)+\gamma Q^\pi(s',a')\right]
\end{align*}$$
Then, since $\pi^\ast(s) = \arg\max_a Q^\ast(s,a)$:
$$\begin{align*}
Q^\ast(s,a)
&= E_{(s',a')\sim p(\cdot\;|\;s,a)}\left[r(s,a)+\gamma Q^\ast(s',a')\right] \\
&= E_{s'\sim \pi^\ast(\cdot\;|\;s,a)}\left[r(s,a)+\gamma Q^\ast(s',\pi^\ast(s'))\right] \\
&= E_{s'\sim \pi^\ast(\cdot\;|\;s,a)}\left[r(s,a)+\gamma \max_{a'}Q^\ast(s',a')\right]
\end{align*}$$
Now, since we want $Q(s,a,\theta) \approx Q^\ast(s,a)$, a necessary condition is that $Q(s,a,\theta) \approx r + \gamma \max_{a'} Q(s',a',\theta)$, hence:
$$\begin{align*}
\mathcal L(\theta\;|\;s,a,s',r) = \lVert r + \gamma \max_{a'} Q(s',a',\theta) - Q(s,a,\theta)\rVert^2
\end{align*}$$

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = [None]*self.max_memory
        self.num_items = 0

    def remember(self, m):
        self.memory[self.num_items % self.max_memory] = m
        self.num_items += 1

    def random_access(self):
        return self.memory[np.random.randint(min(self.num_items, self.max_memory))]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if (e + 1) % 10 == 0:
            env.draw(prefix+str(e+1))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size, epsilon=0.1, memory_size=100, batch_size=16, n_state=2, gamma=0.99):
        super(DQN, self).__init__(epsilon=epsilon)

        # Discount for Q learning
        self.discount = gamma
        
        self.grid_size = grid_size
        
        # Number of states
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return self.model.predict(s[None])[0].argmax()

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        next_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        game_overs = np.zeros(self.batch_size, dtype=bool)
        rewards = np.zeros(self.batch_size)
        actions = np.zeros(self.batch_size, dtype=int)
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            [s, n_s, a, r, go] = self.memory.random_access()
            input_states[i] = s
            next_states[i] = n_s
            actions[i] = a
            game_overs[i] = go
            rewards[i] = r
        target_q = self.model.predict(input_states)
        target_q[np.arange(self.batch_size), actions] = rewards + \
            self.discount * (~game_overs) * self.model.predict(next_states).max(axis=-1)
        target_q = np.clip(target_q, -3, 3)
        
        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1, decay=1e-4, momentum=0.9, 
                 num_layers=3, embed_size=50, activation="tanh", **kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        fcs = [keras.layers.Flatten(input_shape=(5, 5, self.n_state))]
        for i in range(num_layers - 1):
            fcs.append(Dense(embed_size, activation=activation))
        fcs.append(Dense(4))
        model = Sequential(fcs)
        
        model.compile(sgd(lr=lr, decay=decay, momentum=momentum), "mse")
        self.model = model

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon=0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train{}.mp4'.format(epochs_train)))

Epoch 000/300 | Loss 0.0072 | Win/lose count 2.0/3.0 (-1.0)
Epoch 001/300 | Loss 0.0014 | Win/lose count 2.5/0 (2.5)
Epoch 002/300 | Loss 0.0028 | Win/lose count 2.0/1.0 (1.0)
Epoch 003/300 | Loss 0.0050 | Win/lose count 3.0/2.0 (1.0)
Epoch 004/300 | Loss 0.0752 | Win/lose count 2.0/1.0 (1.0)
Epoch 005/300 | Loss 0.0061 | Win/lose count 4.0/2.0 (2.0)
Epoch 006/300 | Loss 0.0025 | Win/lose count 7.0/2.0 (5.0)
Epoch 007/300 | Loss 0.0029 | Win/lose count 7.0/3.0 (4.0)
Epoch 008/300 | Loss 0.0041 | Win/lose count 3.0/7.0 (-4.0)
Epoch 009/300 | Loss 0.0307 | Win/lose count 3.5/3.0 (0.5)
Epoch 010/300 | Loss 0.0101 | Win/lose count 5.0/5.0 (0.0)
Epoch 011/300 | Loss 0.0016 | Win/lose count 6.0/5.0 (1.0)
Epoch 012/300 | Loss 0.0037 | Win/lose count 5.0/1.0 (4.0)
Epoch 013/300 | Loss 0.0059 | Win/lose count 4.0/3.0 (1.0)
Epoch 014/300 | Loss 0.0649 | Win/lose count 12.5/2.0 (10.5)
Epoch 015/300 | Loss 0.0033 | Win/lose count 5.5/2.0 (3.5)
Epoch 016/300 | Loss 0.0060 | Win/lose count 15.0/5.0 

Epoch 138/300 | Loss 0.0017 | Win/lose count 7.5/1.0 (6.5)
Epoch 139/300 | Loss 0.0019 | Win/lose count 10.5/3.0 (7.5)
Epoch 140/300 | Loss 0.0016 | Win/lose count 2.0/1.0 (1.0)
Epoch 141/300 | Loss 0.0016 | Win/lose count 4.5/2.0 (2.5)
Epoch 142/300 | Loss 0.0030 | Win/lose count 13.0/1.0 (12.0)
Epoch 143/300 | Loss 0.0017 | Win/lose count 12.0/1.0 (11.0)
Epoch 144/300 | Loss 0.0011 | Win/lose count 11.0/2.0 (9.0)
Epoch 145/300 | Loss 0.0035 | Win/lose count 11.0/1.0 (10.0)
Epoch 146/300 | Loss 0.0018 | Win/lose count 3.5/0 (3.5)
Epoch 147/300 | Loss 0.0579 | Win/lose count 13.5/1.0 (12.5)
Epoch 148/300 | Loss 0.0006 | Win/lose count 7.0/3.0 (4.0)
Epoch 149/300 | Loss 0.0052 | Win/lose count 8.0/2.0 (6.0)
Epoch 150/300 | Loss 0.0021 | Win/lose count 8.5/2.0 (6.5)
Epoch 151/300 | Loss 0.0017 | Win/lose count 8.0/1.0 (7.0)
Epoch 152/300 | Loss 0.0193 | Win/lose count 1.5/1.0 (0.5)
Epoch 153/300 | Loss 0.0062 | Win/lose count 14.0/3.0 (11.0)
Epoch 154/300 | Loss 0.0015 | Win/lose count 1

Epoch 275/300 | Loss 0.0033 | Win/lose count 8.0/2.0 (6.0)
Epoch 276/300 | Loss 0.0610 | Win/lose count 9.5/3.0 (6.5)
Epoch 277/300 | Loss 0.0031 | Win/lose count 9.5/2.0 (7.5)
Epoch 278/300 | Loss 0.0014 | Win/lose count 13.5/2.0 (11.5)
Epoch 279/300 | Loss 0.0029 | Win/lose count 6.5/2.0 (4.5)
Epoch 280/300 | Loss 0.0606 | Win/lose count 4.0/4.0 (0.0)
Epoch 281/300 | Loss 0.0015 | Win/lose count 10.5/2.0 (8.5)
Epoch 282/300 | Loss 0.0012 | Win/lose count 12.5/1.0 (11.5)
Epoch 283/300 | Loss 0.0019 | Win/lose count 17.5/1.0 (16.5)
Epoch 284/300 | Loss 0.0041 | Win/lose count 10.5/2.0 (8.5)
Epoch 285/300 | Loss 0.0008 | Win/lose count 15.0/5.0 (10.0)
Epoch 286/300 | Loss 0.0020 | Win/lose count 7.5/1.0 (6.5)
Epoch 287/300 | Loss 0.0004 | Win/lose count 13.5/3.0 (10.5)
Epoch 288/300 | Loss 0.0016 | Win/lose count 14.0/2.0 (12.0)
Epoch 289/300 | Loss 0.0009 | Win/lose count 14.0/1.0 (13.0)
Epoch 290/300 | Loss 0.0006 | Win/lose count 5.5/3.0 (2.5)
Epoch 291/300 | Loss 0.0641 | Win/lose c

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args, lr=0.1, decay=1e-4, momentum=0.9, 
                 filters1=20, filters2=50, activation="tanh", **kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential([
            Conv2D(filters1, 3, activation=activation, input_shape=(5, 5, self.n_state)),
            Conv2D(filters2, 3, activation=activation),
            keras.layers.Flatten(),
            Dense(4),
        ])
        
        model.compile(sgd(lr=lr, decay=decay, momentum=momentum), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon=0.1, memory_size=2000, batch_size=32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train{}.mp4'.format(epochs_train)))

Epoch 000/300 | Loss 0.0008 | Win/lose count 3.5/1.0 (2.5)
Epoch 001/300 | Loss 0.0042 | Win/lose count 5.5/4.0 (1.5)
Epoch 002/300 | Loss 0.0022 | Win/lose count 4.0/2.0 (2.0)
Epoch 003/300 | Loss 0.0024 | Win/lose count 4.5/6.0 (-1.5)
Epoch 004/300 | Loss 0.0045 | Win/lose count 4.5/4.0 (0.5)
Epoch 005/300 | Loss 0.0009 | Win/lose count 9.5/3.0 (6.5)
Epoch 006/300 | Loss 0.0769 | Win/lose count 14.5/2.0 (12.5)
Epoch 007/300 | Loss 0.0014 | Win/lose count 5.5/1.0 (4.5)
Epoch 008/300 | Loss 0.0013 | Win/lose count 4.5/5.0 (-0.5)
Epoch 009/300 | Loss 0.0038 | Win/lose count 7.5/4.0 (3.5)
Epoch 010/300 | Loss 0.0035 | Win/lose count 13.0/2.0 (11.0)
Epoch 011/300 | Loss 0.0019 | Win/lose count 3.5/1.0 (2.5)
Epoch 012/300 | Loss 0.0015 | Win/lose count 14.0/3.0 (11.0)
Epoch 013/300 | Loss 0.0065 | Win/lose count 8.5/2.0 (6.5)
Epoch 014/300 | Loss 0.0033 | Win/lose count 10.5/5.0 (5.5)
Epoch 015/300 | Loss 0.0031 | Win/lose count 16.5/3.0 (13.5)
Epoch 016/300 | Loss 0.0017 | Win/lose count 

Epoch 137/300 | Loss 0.0016 | Win/lose count 5.0/1.0 (4.0)
Epoch 138/300 | Loss 0.0018 | Win/lose count 8.5/4.0 (4.5)
Epoch 139/300 | Loss 0.0460 | Win/lose count 7.5/4.0 (3.5)
Epoch 140/300 | Loss 0.0017 | Win/lose count 12.5/1.0 (11.5)
Epoch 141/300 | Loss 0.0010 | Win/lose count 19.0/2.0 (17.0)
Epoch 142/300 | Loss 0.0007 | Win/lose count 12.5/2.0 (10.5)
Epoch 143/300 | Loss 0.0008 | Win/lose count 13.5/2.0 (11.5)
Epoch 144/300 | Loss 0.0010 | Win/lose count 13.0/1.0 (12.0)
Epoch 145/300 | Loss 0.0009 | Win/lose count 17.5/2.0 (15.5)
Epoch 146/300 | Loss 0.0605 | Win/lose count 15.0/4.0 (11.0)
Epoch 147/300 | Loss 0.0636 | Win/lose count 20.5/3.0 (17.5)
Epoch 148/300 | Loss 0.0080 | Win/lose count 13.0/2.0 (11.0)
Epoch 149/300 | Loss 0.0018 | Win/lose count 11.0/3.0 (8.0)
Epoch 150/300 | Loss 0.0022 | Win/lose count 23.0/3.0 (20.0)
Epoch 151/300 | Loss 0.0438 | Win/lose count 18.0/1.0 (17.0)
Epoch 152/300 | Loss 0.0030 | Win/lose count 12.0/1.0 (11.0)
Epoch 153/300 | Loss 0.0021 | W

Epoch 273/300 | Loss 0.0012 | Win/lose count 19.0/7.0 (12.0)
Epoch 274/300 | Loss 0.0007 | Win/lose count 9.0/4.0 (5.0)
Epoch 275/300 | Loss 0.0017 | Win/lose count 8.0/3.0 (5.0)
Epoch 276/300 | Loss 0.0014 | Win/lose count 20.5/1.0 (19.5)
Epoch 277/300 | Loss 0.0032 | Win/lose count 8.0/1.0 (7.0)
Epoch 278/300 | Loss 0.0603 | Win/lose count 13.5/1.0 (12.5)
Epoch 279/300 | Loss 0.0588 | Win/lose count 11.5/2.0 (9.5)
Epoch 280/300 | Loss 0.0016 | Win/lose count 11.0/3.0 (8.0)
Epoch 281/300 | Loss 0.0566 | Win/lose count 5.5/2.0 (3.5)
Epoch 282/300 | Loss 0.0014 | Win/lose count 7.5/5.0 (2.5)
Epoch 283/300 | Loss 0.0017 | Win/lose count 16.0/1.0 (15.0)
Epoch 284/300 | Loss 0.0016 | Win/lose count 14.0/2.0 (12.0)
Epoch 285/300 | Loss 0.0585 | Win/lose count 12.5/3.0 (9.5)
Epoch 286/300 | Loss 0.0008 | Win/lose count 11.0/4.0 (7.0)
Epoch 287/300 | Loss 0.0018 | Win/lose count 7.0/4.0 (3.0)
Epoch 288/300 | Loss 0.0561 | Win/lose count 15.5/5.0 (10.5)
Epoch 289/300 | Loss 0.0014 | Win/lose c

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon=0.1, memory_size=2000, batch_size=32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon=0.1, memory_size=2000, batch_size=32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 6.5/2.0. Average score (4.5)
Win/lose count 11.5/2.0. Average score (7.0)
Win/lose count 3.0/4.0. Average score (4.333333333333333)
Win/lose count 4.5/3.0. Average score (3.625)
Win/lose count 6.5/7.0. Average score (2.8)
Win/lose count 3.0/7.0. Average score (1.6666666666666667)
Win/lose count 3.5/2.0. Average score (1.6428571428571428)
Win/lose count 4.5/1.0. Average score (1.875)
Win/lose count 6.5/4.0. Average score (1.9444444444444444)
Win/lose count 10.0/6.0. Average score (2.15)
Final score: 2.15
Test of the FC
Win/lose count 3.5/6.0. Average score (-2.5)
Win/lose count 3.0/2.0. Average score (-0.75)
Win/lose count 5.0/4.0. Average score (-0.16666666666666666)
Win/lose count 5.0/5.0. Average score (-0.125)
Win/lose count 2.5/4.0. Average score (-0.4)
Win/lose count 2.5/2.0. Average score (-0.25)
Win/lose count 3.0/5.0. Average score (-0.5)
Win/lose count 7.5/8.0. Average score (-0.5)
Win/lose count 7.0/4.0. Average score (-0.1111111111111111)
Win/l

In [16]:
HTML(display_videos('cnn_test{}.mp4'.format(epochs_test)))

In [17]:
HTML(display_videos('fc_test{}.mp4'.format(epochs_test)))

The CNN architecture is clearly superior to the fully connected one, both quantitavely (by looking at the final test error) and qualitatively (by watching the replays). The fully connected architecture does not exploit the spatial structure, and thus has a harder time learning the optimal policy (you can see it not grabbing cheese cells that are in its range and can be reached without stepping on a poisonous cell). It is worth noting that both models have exactly the same number of parameters (5,304), so the comparison is not biased towards one architecture or the other. Both algorithms do not explore the whole map and sometimes get stuck in some areas.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [18]:
def train_explore(agent, env, epoch, prefix='', ε0=1.0, α=0.5):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        agent.set_epsilon(ε0 / ((e + 1) ** α))

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if (e + 1) % 10 == 0:
            env.draw(prefix+str(e+1))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.visited = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board > 0, 0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = self.board[self.x, self.y]
        if train:
            reward += -0.1 * self.visited[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        self.visited[self.x, self.y] = 1
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size, 1),
                                self.position.reshape(self.grid_size, self.grid_size, 1),
                                self.visited.reshape(self.grid_size, self.grid_size, 1)), axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        self.visited = np.zeros((self.grid_size, self.grid_size))
        self.visited[self.x, self.y] = 1

        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size, 1),
                                self.position.reshape(self.grid_size, self.grid_size, 1),
                                self.visited.reshape(self.grid_size, self.grid_size, 1)), axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [19]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon=0.1, memory_size=2000, batch_size=32, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore{}.mp4'.format(epochs_train)))

Epoch 000/300 | Loss 0.0060 | Win/lose count 8.0/28.400000000000055 (-20.400000000000055)
Epoch 001/300 | Loss 0.0048 | Win/lose count 17.0/28.100000000000072 (-11.100000000000072)
Epoch 002/300 | Loss 0.0096 | Win/lose count 16.5/24.500000000000032 (-8.000000000000032)
Epoch 003/300 | Loss 0.0046 | Win/lose count 19.5/17.899999999999988 (1.600000000000012)
Epoch 004/300 | Loss 0.0186 | Win/lose count 13.5/27.600000000000065 (-14.100000000000065)
Epoch 005/300 | Loss 0.0050 | Win/lose count 17.0/18.80000000000001 (-1.8000000000000114)
Epoch 006/300 | Loss 0.0050 | Win/lose count 10.5/23.10000000000003 (-12.60000000000003)
Epoch 007/300 | Loss 0.0050 | Win/lose count 18.5/18.70000000000002 (-0.2000000000000206)
Epoch 008/300 | Loss 0.0073 | Win/lose count 22.0/17.19999999999999 (4.800000000000011)
Epoch 009/300 | Loss 0.0424 | Win/lose count 20.0/18.39999999999999 (1.6000000000000085)
Epoch 010/300 | Loss 0.0043 | Win/lose count 14.0/14.799999999999963 (-0.7999999999999634)
Epoch 011/30

Epoch 093/300 | Loss 0.0291 | Win/lose count 22.0/12.199999999999978 (9.800000000000022)
Epoch 094/300 | Loss 0.0028 | Win/lose count 23.5/15.799999999999974 (7.700000000000026)
Epoch 095/300 | Loss 0.0064 | Win/lose count 21.5/18.100000000000012 (3.399999999999988)
Epoch 096/300 | Loss 0.0501 | Win/lose count 20.5/12.399999999999975 (8.100000000000025)
Epoch 097/300 | Loss 0.0044 | Win/lose count 23.5/11.999999999999977 (11.500000000000023)
Epoch 098/300 | Loss 0.0046 | Win/lose count 33.5/12.599999999999984 (20.900000000000016)
Epoch 099/300 | Loss 0.0077 | Win/lose count 19.5/9.99999999999998 (9.50000000000002)
Epoch 100/300 | Loss 0.0054 | Win/lose count 22.5/13.19999999999998 (9.30000000000002)
Epoch 101/300 | Loss 0.0038 | Win/lose count 23.5/15.39999999999997 (8.10000000000003)
Epoch 102/300 | Loss 0.0062 | Win/lose count 18.5/14.79999999999997 (3.7000000000000295)
Epoch 103/300 | Loss 0.0061 | Win/lose count 19.5/14.799999999999972 (4.700000000000028)
Epoch 104/300 | Loss 0.038

Epoch 186/300 | Loss 0.0065 | Win/lose count 24.0/13.999999999999972 (10.000000000000028)
Epoch 187/300 | Loss 0.0524 | Win/lose count 26.0/8.199999999999987 (17.80000000000001)
Epoch 188/300 | Loss 0.0076 | Win/lose count 16.0/12.499999999999975 (3.500000000000025)
Epoch 189/300 | Loss 0.0018 | Win/lose count 15.0/14.699999999999966 (0.30000000000003446)
Epoch 190/300 | Loss 0.0102 | Win/lose count 21.0/8.899999999999984 (12.100000000000016)
Epoch 191/300 | Loss 0.0065 | Win/lose count 22.5/13.899999999999977 (8.600000000000023)
Epoch 192/300 | Loss 0.0365 | Win/lose count 28.0/14.99999999999998 (13.00000000000002)
Epoch 193/300 | Loss 0.0090 | Win/lose count 13.0/15.499999999999964 (-2.4999999999999645)
Epoch 194/300 | Loss 0.0119 | Win/lose count 22.0/12.999999999999977 (9.000000000000023)
Epoch 195/300 | Loss 0.0034 | Win/lose count 21.5/13.199999999999973 (8.300000000000027)
Epoch 196/300 | Loss 0.0095 | Win/lose count 17.0/17.79999999999999 (-0.79999999999999)
Epoch 197/300 | Los

Epoch 279/300 | Loss 0.0103 | Win/lose count 20.5/13.399999999999975 (7.1000000000000245)
Epoch 280/300 | Loss 0.0091 | Win/lose count 11.5/17.39999999999997 (-5.89999999999997)
Epoch 281/300 | Loss 0.0076 | Win/lose count 18.0/13.59999999999997 (4.4000000000000306)
Epoch 282/300 | Loss 0.0688 | Win/lose count 17.0/13.99999999999997 (3.00000000000003)
Epoch 283/300 | Loss 0.0067 | Win/lose count 14.5/16.499999999999975 (-1.9999999999999751)
Epoch 284/300 | Loss 0.0066 | Win/lose count 24.5/15.399999999999972 (9.100000000000028)
Epoch 285/300 | Loss 0.0093 | Win/lose count 23.0/13.999999999999975 (9.000000000000025)
Epoch 286/300 | Loss 0.0166 | Win/lose count 17.0/13.799999999999967 (3.200000000000033)
Epoch 287/300 | Loss 0.0073 | Win/lose count 19.5/16.79999999999997 (2.7000000000000313)
Epoch 288/300 | Loss 0.0099 | Win/lose count 19.0/15.099999999999973 (3.900000000000027)
Epoch 289/300 | Loss 0.0061 | Win/lose count 26.0/13.899999999999977 (12.100000000000023)
Epoch 290/300 | Loss

In [20]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore9.mp4'))

Win/lose count 21.0/2.0. Average score (19.0)
Win/lose count 19.5/2.0. Average score (18.25)
Win/lose count 10.5/4.0. Average score (14.333333333333334)
Win/lose count 22.5/3.0. Average score (15.625)
Win/lose count 17.0/3.0. Average score (15.3)
Win/lose count 13.0/1.0. Average score (14.75)
Win/lose count 10.0/1.0. Average score (13.928571428571429)
Win/lose count 10.0/1.0. Average score (13.3125)
Win/lose count 19.0/3.0. Average score (13.61111111111111)
Win/lose count 18.5/0. Average score (14.1)
Final score: 14.1


The advantage of exploration is clear here: the model is much better than its no-exploration-bonus counterpart.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [31]:
from tensorflow import nn
def collect_predictions(agent, env, epoch):
    states = []
    actions = []
    
    for e in range(epoch):
        state = env.reset()
        game_over = False

        while not game_over:
            action = agent.act(state)
            prev_state = state
            state, reward, game_over = env.act(action, train=False)

            if not game_over:
                states.append(state)
                actions.append(action)
                
    return np.stack(states), np.stack(actions)

class Imitator(Agent):
    def __init__(self):
        super().__init__()
        
        # Same model as CNN
        model = Sequential([
                    Conv2D(20, 3, activation="tanh", input_shape=(5, 5, 3)),
                    Conv2D(50, 3, activation="tanh"),
                    keras.layers.Flatten(),
                    Dense(4), #, activation="softmax"), # doesn't work because of keras/tf incompatibility
                    keras.layers.Lambda(lambda x : nn.softmax(x)),
                ])
        model.compile(sgd(lr=0.1, decay=1e-4, momentum=0.9), "categorical_crossentropy")
        self.model = model
    
    def fit(self, states, actions):
        self.model.fit(states, keras.utils.to_categorical(actions, num_classes=4), 
                       batch_size=512, epochs=epochs_train)
    
    def act(self, s):
        return self.model.predict(s[None])[0].argmax()
    
imitator = Imitator()
imitator.fit(*collect_predictions(agent, env, epochs_train))
test(imitator, env, epochs_test, prefix='imitator')
HTML(display_videos('imitator{}.mp4'.format(epochs_test)))

Epoch 1/300
60000/60000 [==============================] - 1s 13us/step - loss: 0.1791
Epoch 2/300
60000/60000 [==============================] - 1s 13us/step - loss: 0.0774
Epoch 3/300
60000/60000 [==============================] - 1s 12us/step - loss: 0.0670
Epoch 4/300
60000/60000 [==============================] - 1s 15us/step - loss: 0.0609
Epoch 5/300
60000/60000 [==============================] - 1s 13us/step - loss: 0.0543
Epoch 6/300
60000/60000 [==============================] - 1s 12us/step - loss: 0.0514
Epoch 7/300
60000/60000 [==============================] - 1s 12us/step - loss: 0.0486
Epoch 8/300
60000/60000 [==============================] - 1s 12us/step - loss: 0.0465
Epoch 9/300
60000/60000 [==============================] - 1s 12us/step - loss: 0.0444
Epoch 10/300
60000/60000 [==============================] - 1s 12us/step - loss: 0.0431
Epoch 11/300
60000/60000 [==============================] - 1s 13us/step - loss: 0.0414
Epoch 12/300
60000/60000 [===============

60000/60000 [==============================] - 1s 13us/step - loss: 0.0138
Epoch 95/300
60000/60000 [==============================] - 1s 13us/step - loss: 0.0140
Epoch 96/300
60000/60000 [==============================] - 1s 12us/step - loss: 0.0137
Epoch 97/300
60000/60000 [==============================] - 1s 12us/step - loss: 0.0135
Epoch 98/300
60000/60000 [==============================] - 1s 12us/step - loss: 0.0136
Epoch 99/300
60000/60000 [==============================] - 1s 12us/step - loss: 0.0135
Epoch 100/300
60000/60000 [==============================] - 1s 13us/step - loss: 0.0136
Epoch 101/300
60000/60000 [==============================] - 1s 12us/step - loss: 0.0133
Epoch 102/300
60000/60000 [==============================] - 1s 12us/step - loss: 0.0133
Epoch 103/300
60000/60000 [==============================] - 1s 12us/step - loss: 0.0133
Epoch 104/300
60000/60000 [==============================] - 1s 12us/step - loss: 0.0132
Epoch 105/300
60000/60000 [=============

60000/60000 [==============================] - 1s 13us/step - loss: 0.0110
Epoch 187/300
60000/60000 [==============================] - 1s 13us/step - loss: 0.0111
Epoch 188/300
60000/60000 [==============================] - 1s 14us/step - loss: 0.0112
Epoch 189/300
60000/60000 [==============================] - 1s 13us/step - loss: 0.0111
Epoch 190/300
60000/60000 [==============================] - 1s 12us/step - loss: 0.0110
Epoch 191/300
60000/60000 [==============================] - 1s 12us/step - loss: 0.0111
Epoch 192/300
60000/60000 [==============================] - 1s 12us/step - loss: 0.0110
Epoch 193/300
60000/60000 [==============================] - 1s 13us/step - loss: 0.0109
Epoch 194/300
60000/60000 [==============================] - 1s 13us/step - loss: 0.0111
Epoch 195/300
60000/60000 [==============================] - 1s 13us/step - loss: 0.0109
Epoch 196/300
60000/60000 [==============================] - 1s 13us/step - loss: 0.0110
Epoch 197/300
60000/60000 [========

60000/60000 [==============================] - 1s 13us/step - loss: 0.0103
Epoch 279/300
60000/60000 [==============================] - 1s 15us/step - loss: 0.0103
Epoch 280/300
60000/60000 [==============================] - 1s 13us/step - loss: 0.0103
Epoch 281/300
60000/60000 [==============================] - 1s 13us/step - loss: 0.0103
Epoch 282/300
60000/60000 [==============================] - 1s 13us/step - loss: 0.0104
Epoch 283/300
60000/60000 [==============================] - 1s 13us/step - loss: 0.0104
Epoch 284/300
60000/60000 [==============================] - 1s 14us/step - loss: 0.0103
Epoch 285/300
60000/60000 [==============================] - 1s 13us/step - loss: 0.0103
Epoch 286/300
60000/60000 [==============================] - 1s 12us/step - loss: 0.0103
Epoch 287/300
60000/60000 [==============================] - 1s 12us/step - loss: 0.0104
Epoch 288/300
60000/60000 [==============================] - 1s 12us/step - loss: 0.0103
Epoch 289/300
60000/60000 [========

Given the same data (a set number of epochs), the same architecture, and much more training time (each epoch is done one the whole data instead of a single batch), the imitator does not reach satisfactory performance, whereas the original agent was almost optimal. This is due to the fact that the imitator solves a classification problem, where there is no notion of time and reward, and thus does not model the environment and its transition function. Therefore, it fares much worse in new, previously unseen situations.

***